# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [35]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [36]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [37]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[:5])

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

1718
[['', 'Logged Out', '', '', '0', '', '', 'free', '', 'PUT', 'Login', '', '741', '', '307', '1.54276E+12', ''], ['', 'Logged In', 'Theodore', 'M', '1', 'Smith', '', 'free', 'Houston-The Woodlands-Sugar Land, TX', 'GET', 'Home', '1.54031E+12', '741', '', '200', '1.54276E+12', '52'], ['', 'Logged Out', '', '', '0', '', '', 'paid', '', 'GET', 'Home', '', '774', '', '200', '1.54276E+12', ''], ['', 'Logged Out', '', '', '1', '', '', 'paid', '', 'GET', 'Home', '', '774', '', '200', '1.54276E+12', ''], ['', 'Logged Out', '', '', '2', '', '', 'paid', '', 'PUT', 'Login', '', '774', '', '307', '1.54276E+12', '']]


In [38]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

1421


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [39]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [40]:
# Create a Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION =
    {'class':'SimpleStrategy','replication_factor': 1}"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [41]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
select * from music_event_song_info where sessionId = 338 and itemInSession = 4

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
select * from music_event_song_user_info where userid = 10 and sessionid = 182

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
select * from music_event_user_info where song_title = 'All Hands Against His Own'

#### Creating tables

##### Table1: music_event_song_info
##### Primary Key: session_id, item_in_session (Unique Identifier)
##### Partition Key: session_id, item_in_session

In [42]:
create_query1 = "CREATE TABLE IF NOT EXISTS music_event_song_info"
create_query1 = create_query1 + "(session_id int,  item_in_session int, \
                song_title text, song_artist text, song_length decimal,\
                user_id int, user_first_name text, user_last_name text, user_gender text, user_level text, user_location text,\
                PRIMARY KEY((session_id, item_in_session)))"

try:
    session.execute(create_query1)
except Exception as e:
    print(e)

##### Table2: music_event_song_user_info
##### Primary Key: user_id, session_id, item_in_session
##### Partition Key: user_id, session_id
##### Clustering Key: item_in_session (there is need to sort by itemInSession)

In [43]:
create_query2 = "CREATE TABLE IF NOT EXISTS music_event_song_user_info"
create_query2 = create_query2 + "(user_id int, session_id int, item_in_session int, \
                song_title text, song_artist text, song_length decimal,\
                user_first_name text, user_last_name text, user_gender text, user_level text, user_location text,\
                PRIMARY KEY((user_id, session_id), item_in_session))"

try:
    session.execute(create_query2)
except Exception as e:
    print(e)

##### Table3: music_event_user_info
##### Primary Key: song_title, user_id
##### Partition Key: song_title (Partition key must include in the query; WHERE clause only filters on song_title)
##### Clustering Key: user_id (to make the Primary Key unique)

In [44]:
create_query3 = "CREATE TABLE IF NOT EXISTS music_event_user_info"
create_query3 = create_query3 + "(song_title text, user_id int, \
                session_id int, item_in_session int, \
                song_artist text, song_length decimal,\
                user_first_name text, user_last_name text, user_gender text, user_level text, user_location text,\
                PRIMARY KEY((song_title), user_id))"

try:
    session.execute(create_query3)
except Exception as e:
    print(e)

#### Inserting data into table1: music_event_song_info

In [45]:
from prettytable import PrettyTable

In [50]:
## TO-DO: Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "SELECT song_artist, song_title, song_length FROM music_event_song_info WHERE session_id = 774 AND item_in_session = 4"

In [51]:
# Insert data into table music_event_song_info
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        query = "INSERT INTO music_event_song_info (session_id,  item_in_session, \
                song_title, song_artist, song_length,\
                user_id, user_first_name, user_last_name, user_gender, user_level, user_location)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[-3]), int(line[3]), line[-2], line[0], float(line[5]), int(line[-1]), line[1], line[4], line[2], line[6], line[7]))

#### Validating the data: SELECT to verify that the data have been inserted into table1

In [52]:
query1

'SELECT song_artist, song_title, song_length FROM music_event_song_info WHERE session_id = 774 AND item_in_session = 4'

In [53]:
## SELECT statement to verify the data was entered into the table
try:
    rows = session.execute(query1)
except Exception as e:
    print(e)
    
# use pretty table to display data in tabular form and include headings
t = PrettyTable(['Artist', 'Song', 'Length'])
    
for row in rows:
    t.add_row([row.song_artist, row.song_title, row.song_length])

print(t)

+----------------------------------+-------------------+-----------+
|              Artist              |        Song       |   Length  |
+----------------------------------+-------------------+-----------+
| Facto Delafe y las flores azules | Enero en la playa | 315.81995 |
+----------------------------------+-------------------+-----------+


#### Inserting data into table2: into music_event_song_user_info 
#### Validating the data

In [54]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "SELECT song_artist, song_title, user_first_name, user_last_name FROM music_event_song_user_info WHERE session_id = 671 and user_id = 97"

In [21]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_event_song_user_info (user_id, session_id,  item_in_session, \
                song_title, song_artist, song_length,\
                user_first_name, user_last_name, user_gender, user_level, user_location)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[-1]), int(line[-3]), int(line[3]), line[-2], line[0], float(line[5]), line[1], line[4], line[2], line[6], line[7]))

In [55]:
query2

'SELECT song_artist, song_title, user_first_name, user_last_name FROM music_event_song_user_info WHERE session_id = 671 and user_id = 97'

In [56]:
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
t=PrettyTable(['Artist','Song','FirstName','LastName'])

for row in rows:
    t.add_row([row.song_artist, row.song_title, row.user_first_name, row.user_last_name])

print(t)

+--------+------+-----------+----------+
| Artist | Song | FirstName | LastName |
+--------+------+-----------+----------+
+--------+------+-----------+----------+


#### Inserting data into table3: music_event_user_info
#### Validating the data

In [24]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "SELECT user_first_name, user_last_name FROM music_event_user_info WHERE song_title = 'You Gotta Be'"

In [25]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO music_event_user_info (song_title, user_id, \
                 session_id, item_in_session, \
                 song_artist, song_length,\
                 user_first_name, user_last_name, user_gender, user_level, user_location)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        
        session.execute(query, (line[-2], int(line[-1]), int(line[-3]), int(line[3]), line[0], float(line[5]), line[1], line[4], line[2], line[6], line[7]))

In [26]:
query3

"SELECT user_first_name, user_last_name FROM music_event_user_info WHERE song_title = 'You Gotta Be'"

In [27]:
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

t = PrettyTable(['FirstName','LastName'])

for row in rows:
    t.add_row([row.user_first_name, row.user_last_name])

print(t)

+-----------+----------+
| FirstName | LastName |
+-----------+----------+
|   Kaylee  | Summers  |
+-----------+----------+


### Drop the tables before closing out the sessions

In [28]:
drop_query1= "DROP TABLE music_event_song_info"
drop_query2= "DROP TABLE music_event_song_user_info"
drop_query3= "DROP TABLE music_event_user_info"

In [29]:
for i in range(1,4):
    try:
        query = 'drop_query'+str(i)
        session.execute(eval(query))
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [30]:
session.shutdown()
cluster.shutdown()